In [9]:
import numpy as np

# Define the Bond class to represent individual bonds
class Bond:
    def __init__(self, face_value, coupon_rate, maturity, price, duration):
        self.face_value = face_value
        self.coupon_rate = coupon_rate
        self.maturity = maturity
        self.price = price
        self.duration = duration

    # Revalue the bond based on a new interest rate (simplified formula)
    def revalue(self, new_interest_rate):
        coupon_payment = self.face_value * self.coupon_rate
        cash_flows = np.array([coupon_payment / (1 + new_interest_rate)**t for t in range(1, self.maturity + 1)])
        cash_flows = np.append(cash_flows, self.face_value / (1 + new_interest_rate)**self.maturity)  # Add face value at maturity
        self.price = np.sum(cash_flows)
        return self.price


# Define the FixedIncomePortfolio class
class FixedIncomePortfolio:
    def __init__(self, bonds):
        self.bonds = bonds

    # Revalue only affected bonds based on a new interest rate
    def partial_revaluation(self, new_interest_rate, affected_bonds_indices):
        for idx in affected_bonds_indices:
            self.bonds[idx].revalue(new_interest_rate)

    # Calculate the total value of the portfolio
    def calculate_portfolio_value(self):
        return sum(bond.price for bond in self.bonds)

    # Calculate the portfolio VaR based on historical scenarios
    def calculate_var(self, historical_changes, time_horizon=1):
        # Calculate portfolio value changes across all historical scenarios
        portfolio_changes = [
            np.sum([bond.price * change for bond, change in zip(self.bonds, scenario)])
            for scenario in historical_changes
        ]

        # Print the portfolio changes for debugging
        print("Portfolio changes across historical scenarios:", portfolio_changes)

        # Sort the portfolio changes
        sorted_changes = np.sort(portfolio_changes)
        print(f"Sorted portfolio changes: {sorted_changes}")

        # Calculate VaR as the 5th percentile of the portfolio changes (worst-case at 95% confidence level)
        var = np.percentile(sorted_changes, 5)

        # Scale VaR for the specified time horizon (assuming daily VaR, scale for weekly or monthly)
        # Scaling factor: sqrt(number of trading days in the target time horizon)
        if time_horizon > 1:  # Time horizon > 1 day
            trading_days = 252  # Standard number of trading days in a year
            time_factor = np.sqrt(trading_days * time_horizon / 252)  # scale VaR for the desired time horizon
            var *= time_factor

        return var


# Sample Bond data (face_value, coupon_rate, maturity, price, duration)
bonds = [
    Bond(face_value=1000, coupon_rate=0.05, maturity=5, price=950, duration=4.5),
    Bond(face_value=1000, coupon_rate=0.03, maturity=10, price=1050, duration=8.0),
    Bond(face_value=1000, coupon_rate=0.04, maturity=3, price=980, duration=2.8)
]

# Create the Portfolio
portfolio = FixedIncomePortfolio(bonds)

# Historical changes in bond prices (as relative changes for each bond in the portfolio)
# Example: Scenario 1: +2% in Bond 1, -1% in Bond 2, +3% in Bond 3
historical_changes = [
    [0.02, -0.01, 0.03],  # Scenario 1
    [-0.01, 0.00, 0.02],  # Scenario 2
    [0.01, -0.02, 0.01],  # Scenario 3
    [-0.02, 0.01, 0.00],  # Scenario 4
    [0.03, -0.01, -0.02],  # Scenario 5
]

# Step 1: Calculate the initial portfolio value
initial_value = portfolio.calculate_portfolio_value()
print(f"Initial Portfolio value: {initial_value}")

# Step 2: Perform Partial Revaluation (only update the bond with maturity > 5 years)
affected_bonds_indices = [1]  # Revalue the bond with 10-year maturity
portfolio.partial_revaluation(new_interest_rate=0.04, affected_bonds_indices=affected_bonds_indices)

# Step 3: Portfolio value after partial revaluation
portfolio_value_after_revaluation = portfolio.calculate_portfolio_value()
print(f"Portfolio value after partial revaluation: {portfolio_value_after_revaluation}")

# Step 4: Calculate VaR (using historical simulation) for different time horizons
# Let's calculate VaR for 1 day, 1 week (5 trading days), and 1 month (21 trading days)
var_1_day = portfolio.calculate_var(historical_changes, time_horizon=1)
var_1_week = portfolio.calculate_var(historical_changes, time_horizon=5)
var_1_month = portfolio.calculate_var(historical_changes, time_horizon=21)

print(f"Portfolio Value at Risk (VaR) for 1 day: {var_1_day}")
print(f"Portfolio Value at Risk (VaR) for 1 week: {var_1_week}")
print(f"Portfolio Value at Risk (VaR) for 1 month: {var_1_month}")


Initial Portfolio value: 2980
Portfolio value after partial revaluation: 2848.8910422064496
Portfolio changes across historical scenarios: [39.2110895779355, 10.100000000000001, 0.9221791558710102, -9.811089577935505, -0.2889104220644967]
Sorted portfolio changes: [-9.81108958 -0.28891042  0.92217916 10.1        39.21108958]
Portfolio changes across historical scenarios: [39.2110895779355, 10.100000000000001, 0.9221791558710102, -9.811089577935505, -0.2889104220644967]
Sorted portfolio changes: [-9.81108958 -0.28891042  0.92217916 10.1        39.21108958]
Portfolio changes across historical scenarios: [39.2110895779355, 10.100000000000001, 0.9221791558710102, -9.811089577935505, -0.2889104220644967]
Sorted portfolio changes: [-9.81108958 -0.28891042  0.92217916 10.1        39.21108958]
Portfolio Value at Risk (VaR) for 1 day: -7.906653746761303
Portfolio Value at Risk (VaR) for 1 week: -17.67981525231168
Portfolio Value at Risk (VaR) for 1 month: -36.232839288339875
